In [1]:
import os
from thermof.read import read_log, read_thermo
import numpy as np
from wham import *
import matplotlib.pyplot as plt
%matplotlib inline

## WHAM 1-D
This notebook is for WHAM analysis in 1D.

#### Surface scan parameters

In [ ]:
SCAN = 'HtBDC_Cu110'
SCANDIR = '/home/kutay/Documents/git/Nanocar/surface-scan'
DATADIR = os.path.join(SCANDIR, 'analysis', 'data', SCAN)
WHAMDIR = os.path.join(SCANDIR, 'analysis', 'wham', 'run')
DRX = 'y'
K_SPRING = 200                                 # Spring constant in kcal/mol
TIMESTEP = 1                                   # Timestep in femtoseconds
T_CONVERSION = 1e-3                            # femtosecond -> picosecond
T_SKIP = 5                                     # Skip first 5 ps of simulation for WHAM analysis
HIST = dict(x=(21, 26), y=(22, 26))
hist_min, hist_max = HIST[DRX]

In [ ]:
thermo_headers = 'Step Temp Press TotEng E_pair E_mol Fmax Fnorm c_C1[1] c_C1[2] c_C1[3]'
thermo_keys = ['step', 'temp', 'press', 'etotal', 'epair', 'emol', 'fmax', 'fnorm', 'x', 'y', 'z']

In [ ]:
start =  [23.3805, 24.1629, 10]          # Molecule starting position
xdist = 2                                # Distance to be traveled in y-axis (use 0 for 1D)
ydist = 1.5                              # Distance to be traveled in x-axis (use 0 for 1D)
dx, dy = 0.2, 0.1                        # Step size
buffer = 0.2                             # Buffer distance

xpos = np.arange(start[0] - xdist - buffer, start[0] + xdist + buffer + dx, dx)
ypos = np.arange(start[1] - ydist - buffer, start[1] + ydist + buffer + dy, dy)
print('x: %i points | y: %i points | Total: %i points' % (len(xpos), len(ypos), len(xpos) * len(ypos)))

### Read and separate scans in y-direction
A separate WHAM analysis will be made for scans in y-direction with different starting x values.

In [ ]:
scanlist = {}
for xi, x in enumerate(xpos):
    scanlist[xi] = []
    for scan in os.listdir(DATADIR):
        scan_x = int(scan.split('-')[0])
        if scan_x == xi:
            scanlist[xi].append(os.path.join(DATADIR, scan))

### Write 1D WHAM time series files

In [ ]:
for scanidx in scanlist:
    # Create WHAM directory
    scandir = os.path.join(WHAMDIR, SCAN, str(scanidx))
    tsdir = os.path.join(scandir, 'ts')
    os.makedirs(tsdir, exist_ok=True)
    tsidx = 0
    eq_pos, ts_files = {}, {}
    nsim = len(scanlist[scanidx])
    for simdir in scanlist[scanidx]:
        # Read log file
        logfile = os.path.join(simdir, 'log.%s' % SCAN)
        # simidx, xi, yi = [int(i) for i in os.path.basename(simdir).split('-')]
        xi, yi = [int(i) for i in os.path.basename(simdir).split('-')]
        thermo_data = read_log(logfile, headers=thermo_headers)
        thermo = read_thermo(thermo_data, headers=thermo_keys)[0]
        # Write time series files
        timeseriesfile = os.path.join(tsdir, '%i.dat' % (tsidx))
        # SHIFT TIME AND COORDINATES -----------------------------------------------------------------------------
        time = timesteps_to_time(thermo['step'][T_SKIP:], dt=TIMESTEP, conversion=T_CONVERSION, shift=int(T_SKIP * TIMESTEP))
        coordinates = thermo[DRX][T_SKIP:]
        # --------------------------------------------------------------------------------------------------------
        write_timeseries_file(timeseriesfile, time, coordinates)
        # Record starting position (min energy pos)
        eq_pos[yi] = ypos[yi]
        ts_files[yi] = timeseriesfile
        tsidx += 1
    # Write WHAM input file
    ts_files = [ts_files[i] for i in range(nsim)]
    eq_pos = [eq_pos[i] for i in range(nsim)]
    spring_k = [K_SPRING] * nsim
    datafile = os.path.join(scandir, '%i.in' % (scanidx))
    write_data_file(datafile, ts_files, eq_pos, spring_k)

### Setup WHAM

In [ ]:
bin_size = dy
num_bins = np.ceil((hist_max - hist_min) / bin_size)
tolerance = 1e-5
temperature = 200
numpad = 0
wham_exec = '/home/kutay/Documents/Research/Software/wham/wham/wham'

### Run WHAM

In [ ]:
SCAN_DATA = {}
for scanidx in scanlist:
    scandir = os.path.join(WHAMDIR, SCAN, str(scanidx))
    datafile = os.path.join(scandir, '%i.in' % (scanidx))
    outfile = os.path.join(scandir, '%i.out' % (scanidx))
    wham_args = [wham_exec, hist_min, hist_max, num_bins, tolerance, temperature, numpad, datafile, outfile]
    data = run_wham(wham_args, verbose=False)
    SCAN_DATA[scanidx] = data

## Plot energy barriers

In [ ]:
def subplot(plot_fun, plot_args, nrow=1, width=3, height=3, dpi=200, save=None):
    n_plots = len(plot_args)
    ncol = np.ceil(n_plots / nrow)
    figsize = (ncol * width, nrow * height)
    fig = plt.figure(figsize=figsize, dpi=dpi)
    fig.subplots_adjust(hspace=.5, wspace=.25)
    for idx, args in enumerate(plot_args, start=1):
        args['ax'] = fig.add_subplot(nrow, ncol, idx)
        plot_fun(**args)
    if save is not None:
        plt.savefig(save, dpi=dpi, transparent=True, bbox_inches='tight')
        
def wham_plot(x, y, ax, xlabel, ylabel, title, xlim):
    ax.plot(x, y, '-o', c='xkcd:crimson', lw=2, markersize=6)
    plt.xlim(xlim)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

In [ ]:
fplot = []
for scan_idx in SCAN_DATA:
    data = SCAN_DATA[scan_idx]
    fplot.append(dict(x=data['coor'], y=data['free'], xlabel='%s (Å)' % DRX, ylabel='Free energy (kcal/mol)', title='%s | %i : %.2f' % (SCAN, scan_idx, xpos[scan_idx]), xlim=HIST[DRX]))

In [ ]:
plt_file = os.path.join(SCANDIR, 'analysis', 'wham', 'plots', '%s-barriers.png' % SCAN)
subplot(wham_plot, fplot, width=6, dpi=300, nrow=5, save=plt_file)

## Save data
Save WHAM data for surface plot using Bokeh

In [ ]:
import yaml 
scan_yaml = {'x': [], 'y': [], 'z': []}
for scanidx in range(len(scanlist)):
    data = SCAN_DATA[scanidx]
    scan_x_pos = [xpos[scanidx]] * len(data['coor'])
    scan_yaml['x'] += scan_x_pos
    scan_yaml['y'] += data['coor']
    scan_yaml['z'] += data['free']

SURFDIR = os.path.join(SCANDIR, 'analysis', 'surface-plot', 'data')
with open(os.path.join(SURFDIR, '%s-data.yaml' % SCAN), 'w') as yf:
    yaml.dump(scan_yaml, yf)
print('Done!')

### 3D Energy Plot

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt

In [ ]:
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
cmap = plt.get_cmap('viridis')
max_energy = 12

for scanidx in range(len(scanlist)):
    data = SCAN_DATA[scanidx]
    scan_x_pos = [xpos[scanidx]] * len(data['coor'])
    F = np.array(data['free'])
    F[np.isinf(F)] = max_energy
    ax.scatter(data['coor'], scan_x_pos, data['free'], '-o', c=[cmap(i) for i in (F / max(F))], lw=2, s=100, alpha=1, zorder=len(scanlist) - scanidx)
    # ax.plot(data['coor'], scan_x_pos, data['free'], '-o', c=[cmap(i) for i in (F / max(F))], lw=2, markersize=6, alpha=1, zorder=len(scanlist) - scanidx)
    # ax.xlim(HIST[DRX])

# Plot surface atoms
surfatoms1 = [[21.582, 22.8912], [25.179, 22.8912], [21.582, 25.4346], [25.179, 25.4346]]
surfatoms2 = [23.3805, 24.1629]
for satom in surfatoms1:
    ax.scatter(satom[1], satom[0], 0, s=700, c='k')
ax.scatter(surfatoms2[1], surfatoms2[0], -1, s=700, c='k', alpha=0.8)

plt.title('WHAM Energy Barrier | %s' % DRX)
ax.set_xlabel('y (Å)')
ax.set_ylabel('x (Å)')
ax.set_zlabel('Free energy (kcal/mol)')
plt.show()